In [1]:
import sys
import pandas as pd
from collections import defaultdict
import spacy
#from spacy.matcher import PhraseMatcher
import math   
from spacy.util import minibatch , compounding
import random
import torch
import numpy as np
from progressbar import ProgressBar, Bar, Percentage
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from spacy_transformers.util import cyclic_triangular_rate
from thinc.neural.optimizers import Adam
from thinc.neural import Model

Q:\anaconda\envs\myEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
Q:\anaconda\envs\myEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
Q:\anaconda\envs\myEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
Q:\anaconda\envs\myEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type

In [2]:
train_data_full = pd.read_csv("train_data.csv")
train_target_full = pd.read_csv("train_target.csv")

In [3]:
features_dict = {}
for author, group in train_data_full.groupby("author"):
    features_dict[author] = " ".join(group["subreddit"].unique())
print(len(features_dict))

5000


In [4]:
to_be_dfed = {"author": list(features_dict.keys()) , "subreddit" : list(features_dict.values()) }
author_subrdts = pd.DataFrame.from_dict(to_be_dfed)
print(author_subrdts.head(5), len(author_subrdts))

           author                                          subreddit
0          -Jared                       AskReddit tall pics StarWars
1         -Peeter                                             gainit
2        -evasian  MouseReview MechanicalKeyboards jailbreak AskR...
3         -rubiks                                    AskWomen AskMen
4  -true_neutral-                    mildlyinteresting todayilearned 5000


In [5]:
gender = [0 for i in range(len(author_subrdts))]

for idx, row in train_target_full.iterrows():
    if row.gender == 1:
        indexes = author_subrdts.index[author_subrdts["author"] == row.author].tolist()
        for i in indexes:
                #print("inside the for")
                gender[i] += 1

if(len(np.unique(gender) == 2)):
    print("ok")
else:
    print("there has been an error with gender recognition, please halt the program now")

print("len(gender) = ", len(gender))

ok
len(gender) =  5000


In [6]:
author_subrdts["gender"] = gender
print(author_subrdts.head(15))

             author                                          subreddit  gender
0            -Jared                       AskReddit tall pics StarWars       0
1           -Peeter                                             gainit       0
2          -evasian  MouseReview MechanicalKeyboards jailbreak AskR...       0
3           -rubiks                                    AskWomen AskMen       1
4    -true_neutral-                    mildlyinteresting todayilearned       0
5           -willis                       tipofmytongue ifyoulikeblank       0
6          00708070                                myfriendwantstoknow       0
7           0200008                                          AskReddit       1
8         05Lanky05                                  trees bestof IAmA       0
9   0urlittlesecret  gonewild dirtypenpals AdviceAnimals gifs world...       0
10        0utlander  totalwar AskReddit gameofthrones MapPorn Fallo...       0
11            0xJRS  Nootropics aww Entrepreneur gol

In [7]:
# balanced
author_f = author_subrdts[author_subrdts["gender"] == 1]
author_m = author_subrdts[author_subrdts["gender"] == 0]
print("len(author_f) = ", len(author_f))
seed = 100
split = math.floor(len(author_f)*1)

train_f = author_f.sample(n = split, random_state = seed)
#test_f = author_f.drop(train_f.index)

train_m = author_m.sample(n  = split, random_state = seed)
#test_m = author_m.drop(train_m.index)

train_subs_f = train_f["subreddit"].tolist()
train_subs_m = train_m["subreddit"].tolist()
#test_sub_f = test_f["subreddit"].tolist()
#test_subs_m = test_m["subreddit"].tolist()

train_texts = train_subs_f + train_subs_m
train_labels = [{'cats': {'1': True ,'0': False}} for label in range(split)] + [{'cats': {'1': False ,'0': True }} for label in range(split)]
#test_texts = test_subs_f + test_subs_m
#test_labels = [1 for i in range(split)] + [0 for i in range(split)]

train_data = list(zip(train_texts, train_labels))
print("len(train_data) = ", len(train_data))

# end of balanced

len(author_f) =  1349
len(train_data) =  2698


In [8]:
# unbalanced
# train_texts  = author_subrdts["subreddits"].tolist()
# train_labels = [{'cats': {'1': label == 1,'0': label == 0}} for label in author_subrdts["gender"].tolist()]
# train_data  = list(zip(train_texts, train_labels))
# print("len(train_data) = ", len(train_data))

# end unbalanced

In [9]:
# carica un learner che parla inglese
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" (bag of words) architecture.
if 'textcat' not in nlp.pipe_names:
    textcat = nlp.create_pipe(
                  "textcat",
                  config={"exclusive_classes": True, "architecture": "bow" }) #https://github.com/explosion/spaCy/issues/3611 #, "n_gram_size" : 2, "attr": "lower" 
    nlp.add_pipe(textcat)
else:
    textcat = nlp.get_pipe('textcat')

# create labels (do they coomute)
textcat.add_label("1")
textcat.add_label("0")




# add the textcat to the spacy pipe

#nlp.add_pipe(textcat)

1

In [10]:
%%time

random.seed(1)
spacy.util.fix_random_seed(1)

# train_data for unique suvredidts, trian_data_nu for not unique, train_data_nuau for not unique in train and unique in test
train_data = train_data
# write it otherwise it is not happy
#nlp.vocab.vectors.name = 'spacy_pretrained_vectors'

# speaks for itself
print("random seeds set")

losses = {}
rocs = []
run_title = "RUN : spacy Ensemble Lemmatized Averaged drop = 0.1 , compound(500,1000,1.001), ngram_size = 2  \n"
output = ""

print("strings and lists initialized")

f_w = sum([1 if y == 1 else 0 for y in  author_subrdts["gender"].tolist()])
m_w = sum([1 if y == 0 else 0 for y in  author_subrdts["gender"].tolist()])

#learning process
#for batch_size in [10, 50, 100, 200, 300] : 
pipe_exceptions = ['textcat'] #( ,"trf_wordpiecer", "trf_tok2vec" ) are required, #https://github.com/explosion/spaCy/blob/master/examples/training/train_textcat.py
print("pipe_exceptions defined")
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
print("other_pipes defined")
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training(component_cfg = {"textcat":{"weights": [1./f_w, 1./m_w]}})
#     batch_size  = 4
#     learn_rate = 0.0005
#     learn_rates = cyclic_triangular_rate(learn_rate / 3, learn_rate * 3, 2 * len(train_data) // batch_size)
#     nlp.begin_training()
#     ops = Model.ops
#     beta1 = 0.9 #0.9
#     beta2 = 0.999 #0.999
#     eps = 1e-10 #1e-8
#     L2 = 0.0 #1e-6
#     max_grad_norm = 1 #1.0
#     optimizer = Adam(ops, learn_rate, L2=L2, beta1=beta1, beta2=beta2, eps=eps)
#     optimizer.max_grad_norm = max_grad_norm
#     optimizer.device = ops.device
    for epoch in range(3):
        print("shuffling data...")
        random.shuffle(train_data)
        #optimizer.learn_rate = float(next(learn_rates))
        # Create the batch generator with batch size = 8
        print("creating batches...")
        batches = minibatch(train_data, size=compounding(4, 32, 1.001)) # compounding(500., 3000., 1.001)  compounding(4, 32, 1.001)
        # Iterate through minibatches
        pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=2000).start()  #8387 is the total number of iterations needed for an unbalanced 0.8 trainset. 0.04 of unbalanced would be approx 1540 . 4595 for lemmatized balanced. 8420 for lemmatized unbalanced
        i = 0
        print("training...")
        for batch in batches:
            # Each batch is a list of (text, label) but we need to
            # send separate lists for texts and labels to update().
            # This is a quick way to split a list of tuples into lists
            texts1, labels = zip(*batch)
            nlp.update(texts1, labels, sgd=optimizer, losses=losses, drop = 0.1) #, drop = 0.2
            i += 1
            pbar.update(i)
        pbar.finish()
        print("i = ",  i)
#         with textcat.model.use_params(optimizer.averages):
#             rocs.append(evaluate_roc(nlp, textcat))
#             output += f"    epoch = {epoch}, losses = {losses}, roc = {rocs[-1]} \n "
#             print( "epoch = ",epoch," losses = ", losses, "roc = " , rocs[-1] , "i = ", i)
            
# sys.stdout = open("spacyEnsembleLemmatizedAveraged_output_sys.txt", "a")
# print(output)
# sys.stdout.close()
        
# with open("spacyEnsembleLemmatizedAveraged_output_file.txt", "a") as f:
#     f.write(run_title + output)
#     f.close()

# # save the model
# output_dir = %pwd
# nlp.to_disk("ensembleLA")
# print("Saved model to", output_dir)

#load it
# print("Loading from", output_dir)
#nlp2 = spacy.load(output_dir)

random seeds set
strings and lists initialized
pipe_exceptions defined
other_pipes defined
shuffling data...
creating batches...
training...


100%|#########################################################################|


i =  555
shuffling data...
creating batches...
training...


100%|#########################################################################|


i =  555
shuffling data...
creating batches...
training...


i =  555
Wall time: 43.1 s


In [11]:
with textcat.model.use_params(optimizer.averages):
    nlp.to_disk(r'Q:\tooBigToDrive\data-mining\kaggle\my_models\spaCy\savedModels\subs_bal')
    print("saved")

saved


In [12]:
# # to be continued with prediction part
# import spacy
# import pandas as pd

nlp = spacy.load(r'ubs_bal')
print("loaded nlp")
textcat = nlp.get_pipe("textcat")
print("obtained textcat")
df = pd.read_csv("lPunctNumStopLemOovAggTest.csv")
print("loaded df")
docs = [nlp.tokenizer(tex) for tex in df["subreddit"].tolist()]
print("docs done")
scores, a = textcat.predict(docs)
print("done")
print(scores[0])
pred_y = [s[0] for s in scores]
print(pred_y[0:15])
subs_test_predictions = pd.DataFrame({"author": df["author"].tolist(), "gender": pred_y})
print(subs_test_predictions.head(5))
subs_test_predictions.to_csv(r"subs_bal_test_predictions" , index = False)

loaded nlp
obtained textcat
loaded df
docs done
done
[0.48052195 0.519478  ]
[0.48052195, 0.51025856, 0.2669244, 0.40504095, 0.43180564, 0.63537675, 0.48134655, 0.3874973, 0.22327794, 0.007003122, 0.042660423, 0.649002, 0.5144257, 0.2682138, 0.43009955]
           author    pred_y
0    --redbeard--  0.480522
1       -Allaina-  0.510259
2  -AllonsyAlonso  0.266924
3          -Beth-  0.405041
4        -Greeny-  0.431806


In [13]:
#weights = [1./f_w, 1./m_w]
#[0.37892216, 0.39194405, 0.18831716, 0.32629508, 0.31537768, 0.45214972, 0.41050243, 0.26181972, 0.19685002, 0.017932957, 0.05777456, 0.4633669, 0.36835364, 0.24319068, 0.30869114]

In [14]:
#nothing
#[0.37892216, 0.39194405, 0.18831716, 0.32629508, 0.31537768, 0.45214972, 0.41050243, 0.26181972, 0.19685002, 0.017932957, 0.05777456, 0.4633669, 0.36835364, 0.24319068, 0.30869114]